In [ ]:
import os
import glob
import numpy as np
import rasterio
from rasterio.transform import from_origin
import matplotlib.pyplot as plt
from netCDF4 import Dataset
from collections import Counter

# === CONFIG ===
DATA_DIR = r"E:\GOES-R Lightning Data\WEST-2023"
OUTPUT_DIR = r"E:\GOES-R Lightning Data\WEST-RASTERS"
PIXEL_SCALE = 1e5  # scale factor so small values survive GeoTIFF save
THRESHOLD = 1.2e-05
FILE_SUFFIX = "_2023"

max_raster = None

files = glob.glob(os.path.join(DATA_DIR, "*.nc"))

for i, f in enumerate(files, start=1):
    with Dataset(f, "r") as ds:

        energy = ds.variables["Total_Optical_energy"][:].filled(0)

        # skip if all zeros
        if np.all(energy == 0):
            continue

        # initialize arrays on first pass
        if max_raster is None:
            shape = energy.shape
            max_raster = np.zeros(shape, dtype=np.float64)
            mean_sum = np.zeros(shape, dtype=np.float64)
            mean_count = np.zeros(shape, dtype=np.float64)

        max_raster = np.maximum(max_raster, energy)

        mask = energy > THRESHOLD
        mean_sum += np.where(mask, energy, 0)
        mean_count += mask.astype(np.float64)

mean_raster = np.divide(mean_sum, mean_count, out=np.zeros_like(mean_sum), where=mean_count > 0)

max_raster *= PIXEL_SCALE
mean_raster *= PIXEL_SCALE

os.makedirs(OUTPUT_DIR, exist_ok=True)

def save_raster(arr, name):
    out_path = os.path.join(OUTPUT_DIR, f"{name}.tif")
    height, width = arr.shape
    transform = from_origin(0, 0, 1, 1)
    with rasterio.open(
        out_path,
        "w",
        driver="GTiff",
        height=height,
        width=width,
        count=1,
        dtype="float32",
        crs=None,
        transform=transform,
    ) as dst:
        dst.write(arr.astype(np.float32), 1)

save_raster(max_raster, f"max_energy{FILE_SUFFIX}")
save_raster(mean_raster, f"mean_energy{FILE_SUFFIX}")

# === Plot ===
#fig, axes = plt.subplots(1, 5, figsize=(20, 5))
#titles = ["Max",  "Mean"]
#arrays = [max_raster, mean_raster]

#for ax, arr, title in zip(axes, arrays, titles):
#    im = ax.imshow(arr, cmap="inferno")
#    ax.set_title(title)
#    plt.colorbar(im, ax=ax, shrink=0.6)

#plt.tight_layout()
#plt.show()

print("DONE")
